<a href="https://colab.research.google.com/github/CFVV/MSc-assignments/blob/main/2022-2_Computer_Audition/T8_classification%20of%20sounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 8: Clasificación

### Nico y Cami

# Pregunta 1
### Explique el método  de clasificación K-Nearest Neighbors 'KNN'



KNN es un algoritmo de aprendizaje supervisado que se basa en calcular los K-Vecinos más cercanos utilizando alguna función de distancia/similaridad entre los datos con los que fue entrenado y el ejemplo por el que se le está consultando la predicción. KNN predice, en el caso de clasificación, la clase mayoritaria presente entre los k-vecinos encontrados con mayor cercanía al ejemplo de consulta, en este algoritmo el número de vecinos a buscar al momento de hacer una predicción debe ser fijado previamente por lo que corresponde a un  hiperparámetro.

# Pregunta 2
### Explique el método de clasificación Support Vector Machine 'SVM'

SVM es un algoritmo de aprendizaje supervisado propuesto como mejora a algoritmos de máximo márgen, para clasificación este algoritmo se basa en encontrar el hiperplano separador entre dos clases que maximiza la distancia de cada clase al hiperplano. Para encontar el hiperplano separador óptimo, las SVM utilizan aquellos ejemplos de cada clase más "conflictivos", es decir, aquellos ejemplos cuyos atributos los hagan difíciles de distinguir con respecto a los ejemplos de la otra clase, a estos ejemplos que se tienden a encontrar más cercanos al punto intermedio entre las clases se les conoce como "vectores de soporte", el nombre proviene de la forma en la que son utilizados: Para calcular el hiperplano se obtiene el vector, perpendicular al hiperplano candidato, que va desde la proyección del ejemplo que es vector de soporte al hiperplano candidato mismo, obteniendose finalmente que el hiperplano es determinado por el conjunto de estos vectores (vectores de soporte). Como paso previo al ajuste del hiperplano las SVM aplican una tansformación sobre el espacio de atributos mediante una función de kernel que tiene como objetivo aumentar la cantidad de dimensiones en las que se representa a los atributos, con el objetivo de tener mayores posibilidades de encontar un hiperplano separador en este nnuevo espacio (el espacio de la transformación de kernel).

# Pregunta 3
### Explique qué es la matriz de confusión y la medida de desempeño "accuracy"


La matriz de confusión es una matriz que resume el desempeño de un clasificador sobre un conjunto de datos, mediante la sumarización de cuatro métricas, obtenidas al comparar la predicción realizada por el clasificador para cada ejemplo del conjunto sobre el que se le pidieron las predicciónes. Estas cuatro métricas, para clasificación binaria, son como siguen:
> * Verdaderos positivos (TP): Cantidad de veces que el clasificador predijo que un determinado ejemplo era de la clase positiva (marcada como 1 u otra codificación equivlente) y en efecto el ejemplo pertenecía a esa clase.
> * Verdaderos negativos (TN): Cantidad de veces que el clasificacdor predijo que un determinado ejemplo no era de la clase positiva (equivalentemente, predijo entonces que es de la clase negativa: 0) y en efecto dicho ejemplo no pertenecía a la clase positiva.
> * Falsos positivos (FP): cantidad de veces que el clasificador predijo que un ejemplo pertenecía a la clase positiva (1) pero el ejemplo no pertenecía a esta clase.
> * Falsos negativos (FN): Cantidad de veces que el clasificador predijo que un ejemplo pertenecía a la clase negativa (0) pero el ejemplo si pertenecía a esta clase.

Los valores que interesa ver maximizados son los Verdaderos positivos/negativos (aciertos del modelo en predecir la correspondiente clase) encontrados en la diagonal principal de la matriz, mientras que es de interés minimizar aquellos fuera de la diagonal (falsos positivo/negativos).

Para el caso de clasificacion multiclase la matriz de confusión se calcula equivalentemente considerando a cada clase por separado de las demás, obteniendose de forma equivalente, que lo que interesa es maximizar los valores en la diagonal principal (aciertos en predecir una determinada clase) y minimizar los valores en las matrices triangulares inferiores y superiores.

La métrica de _accuracy_, calculada a partir de la matriz de confusion, resume la proporción de aciertos totales realizados por el modelo en el universo de predicciones realizadas (por lo tanto esta métrica tiene su rango en el intervalo $[0,1]$). Se calcula mediante la división:
$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

# Pregunta 4

### Clasificando sonidos percusivos
En la carpeta  'Percussive' hay sonidos percusivos: 'Kicks' y 'Snares'.
Utilizando los descriptores temporales que considere pertinentes clasifique dichos sonidos.
Utilice las muestras de las carpeta 'train' para entrenar y las de la carpeta 'test' para evaluar.
Para cada clasificador entregue la matriz de confusión y la accuracy.

* Utilice KNN con 3 vecinos.
* Utilice SVM, con los parámetros que desee.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from scipy.io import wavfile
from glob import glob
from librosa.feature import spectral_centroid, spectral_flatness
from librosa.onset import onset_strength
from librosa import time_to_samples, frames_to_time
from librosa.feature import mfcc
import librosa
from   tqdm.auto import tqdm
import warnings
import os
from sklearn import svm
warnings.filterwarnings('ignore')


def zero_crossing_rate(wavedata, block_length, samplerate):

    # how many blocks have to be processed?
    num_blocks = int(np.ceil(len(wavedata)/block_length))

    # when do these blocks begin (time in seconds)?
    timestamps = (np.arange(0,num_blocks - 1) * (block_length / float(samplerate)))

    zcr = []

    for i in range(0,num_blocks-1):

        start = i * block_length
        stop  = np.min([(start + block_length - 1), len(wavedata)])

        zc = 0.5 * np.mean(np.abs(np.diff(np.sign(wavedata[start:stop]))))
        zcr.append(zc)

    return np.asarray(zcr)

def get_spectralFlux(segmento, sr = 44100, hop_length = 1024):
    odf_default = onset_strength(y=segmento, sr=sr, n_fft = 1024, hop_length=hop_length)
    frame_time = frames_to_time(np.arange(len(odf_default)),
                                    sr=sr,
                                    hop_length=hop_length)
    return frame_time, odf_default

def count_files(type, echo=False):
    files = 0
    train_paths = ['Percussive/TrainKicks/', 'Percussive/trainSnares/']
    test_paths = ['Percussive/testKicks/', 'Percussive/testSnares/']
    path_dict = {'train': train_paths,'test': test_paths}
    paths = path_dict[type]
    for path in paths:

        for filename in os.listdir(path):
            f = os.path.join(path, filename)
            # checking if it is a file
            if os.path.isfile(f):
                if echo:
                    print(f)
                files += 1
    print(files)
    return files

def get_features(type):
    file_cnt = count_files(type)
    features = 2
    X = np.zeros((file_cnt, features))
    y_gt = np.zeros((file_cnt,))
    train_paths = ['Percussive/TrainKicks/', 'Percussive/trainSnares/']
    test_paths = ['Percussive/testKicks/', 'Percussive/testSnares/']
    path_dict = {'train': train_paths,'test': test_paths}
    t = 0
    data_path = path_dict[type]
    for i in range(len(data_path)):
        path = data_path[i]
        dirs = os.listdir(path)
        print(f'Obteniendo {features} features de {path}')
        for j in tqdm(range(len(dirs))):
            filename = dirs[j]
            f = os.path.join(path, filename)
            # checking if it is a file
            if os.path.isfile(f):
                y, sr = librosa.load(f)
                kwargs = {'y': y, 'sr': sr, 'n_fft': 256}
                cent = librosa.feature.spectral_centroid(**kwargs)
                # spec_bw = librosa.feature.spectral_bandwidth(**kwargs)
                # flatness = librosa.feature.spectral_flatness(y=y, n_fft=256)
                # rolloff = librosa.feature.spectral_rolloff(**kwargs, roll_percent = 0.99)
                zero_crossing = librosa.feature.zero_crossing_rate(y)

                X[t, 0] = np.mean(cent)
                X[t, 1] = np.mean(zero_crossing)

                y_gt[t] = i
                t += 1
    return X, y_gt

### Funciones de ayuda de clasificadores

In [ ]:
def ClassifierKNN(Xtrain,ytrain,Xtest,ytest, neighbors):
  Xtrain = np.array(Xtrain)
  ytrain = np.array(ytrain)
  Xtest = np.array(Xtest)
  ytest = np.array(ytest)
  print(f'Clasificador KNN de {neighbors} vecinos')
  knn = KNeighborsClassifier(n_neighbors=neighbors)
  knn.fit(Xtrain, ytrain)
  ypred        = knn.predict(Xtest)
  print(ypred.shape)
  print(ytest.shape)
  acc          = accuracy_score(ytest, ypred)
  print('Entrenando con '+str(Xtrain.shape[0])+' muestras y probando con '+str(Xtest.shape[0])+' muestras')
  print('Testing Accuracy = '+str(acc*100)+'%')
  C = confusion_matrix(ytest,ypred)
  print('Matriz de Confusión:')
  print(C)
  print("KNN - 3 vecinos")
  print(classification_report(ytest, ypred))
  return acc, C, ypred

def ClassifierSVM(Xtrain, ytrain, Xtest, ytest):
    print('Clasificador SVM')
    clf = svm.SVC()
    clf.fit(Xtrain, ytrain)
    ypred = clf.predict(Xtest)
    acc          = accuracy_score(ytest,ypred)
    print('Entrenando con '+str(Xtrain.shape[0])+' muestras y probando con '+str(Xtest.shape[0])+' muestras')
    print('Testing Accuracy = '+str(acc*100)+'%')
    C = confusion_matrix(ytest,ypred)
    print('Matriz de Confusión:')
    print(C)
    print("SVM")
    print(classification_report(ytest, ypred))
    return acc, C, ypred

Usando **Zero Crossing Rate**

In [ ]:
path = ""
X_kicks_train_pre = [wavfile.read(data_path) for data_path in glob(path + 'Percussive/TrainKicks/*.wav')]
X_kicks_test_pre = [wavfile.read(data_path) for data_path in glob(path +'Percussive/testKicks/*.wav')]
X_snares_train_pre = [wavfile.read(data_path) for data_path in glob(path + 'Percussive/trainSnares/*.wav')]
X_snares_test_pre = [wavfile.read(data_path) for data_path in glob(path +'Percussive/testSnares/*.wav')]

Extracción de características

In [ ]:
max_length = np.max(np.max([array.shape[0] for array in [x[1] for x in np.vstack([X_kicks_train_pre,
                                                                                  X_kicks_test_pre,
                                                                                  X_snares_train_pre,
                                                                                  X_snares_test_pre])]]))

X_kicks_train = np.vstack([zero_crossing_rate(np.pad(x[1], (0,max_length-x[1].shape[0])), 80, x[0]) for x in X_kicks_train_pre])
X_kicks_test = np.vstack([zero_crossing_rate(np.pad(x[1], (0,max_length-x[1].shape[0])), 80, x[0]) for x in X_kicks_test_pre])
X_snares_train = np.vstack([zero_crossing_rate(np.pad(x[1], (0,max_length-x[1].shape[0])), 80, x[0]) for x in X_snares_train_pre])
X_snares_test = np.vstack([zero_crossing_rate(np.pad(x[1], (0,max_length-x[1].shape[0])), 80, x[0]) for x in X_snares_test_pre])

Y_train = ['kick']*X_kicks_train.shape[0] + ['snare']*X_snares_train.shape[0]
Y_test = ['kick']*X_kicks_test.shape[0] + ['snare']*X_snares_test.shape[0]

X_train = np.vstack((X_kicks_train, X_snares_train))
X_test = np.vstack([X_kicks_test, X_snares_test])

X_train, Y_train = shuffle(X_train, Y_train, random_state=15)
X_test, Y_test = shuffle(X_test, Y_test, random_state=15)

## Entrenamiento de los modelos

### KNN - 3 vecinos

In [ ]:
knn_results = ClassifierKNN(X_train, Y_train, X_test, Y_test, 3)

Clasificador KNN de 3 vecinos
(19,)
(19,)
Entrenando con 93 muestras y probando con 19 muestras
Testing Accuracy = 100.0%
Matriz de Confusión:
[[10  0]
 [ 0  9]]
KNN - 3 vecinos
              precision    recall  f1-score   support

        kick       1.00      1.00      1.00        10
       snare       1.00      1.00      1.00         9

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19



### SVM - Parámetros por defecto

In [ ]:
svm_results = ClassifierSVM(X_train, Y_train, X_test, Y_test)

Clasificador SVM
Entrenando con 93 muestras y probando con 19 muestras
Testing Accuracy = 100.0%
Matriz de Confusión:
[[10  0]
 [ 0  9]]
SVM
              precision    recall  f1-score   support

        kick       1.00      1.00      1.00        10
       snare       1.00      1.00      1.00         9

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19



Ambos modelos obtienen una clasificación perfecta. Utilizando Zero Crossing Rate como descriptor tanto una SVM con kernel rbf como KNN con 3 vecinos son capaces de distinguir por completo las dos clases para el conjunto de pruebas.

# Pregunta 5
### Incluyendo además descriptores frecuenciales, clasifique utilizando KNN.
Comente los tres resultados obtenidos.

In [ ]:
X_train_spectral_pre = [x[1].astype(float) for x in np.vstack((X_kicks_train_pre, X_snares_train_pre))]
X_test_spectral_pre = [x[1].astype(float) for x in np.vstack([X_kicks_test_pre, X_snares_test_pre])]

max_length = np.max(np.max([array.shape[0] for array in [x for x in X_train_spectral_pre+X_test_spectral_pre]]))

X_train_spectralCentroid  = np.vstack([spectral_centroid(np.pad(x, (0,max_length-x.shape[0])).astype(float), sr = 44100, n_fft=64, hop_length=64)[0] for x in X_train_spectral_pre])
X_test_spectralCentroid = np.vstack([spectral_centroid(np.pad(x, (0,max_length-x.shape[0])).astype(float), sr = 44100, n_fft=64, hop_length=64)[0] for x in X_test_spectral_pre])

In [ ]:
# Por ahora solo probare con centroide espectral
#X_train_spectralFlatness= [spectral_flatness(x, n_fft=8, hop_length=8)[0] for x in X_train]
#X_test_spectralFlatness= [spectral_flatness(x, n_fft=8, hop_length=8)[0] for x in X_test]
#
#X_train_spectralFlux= [get_spectralFlux(x, hop_length=8)[0] for x in X_train]
#X_test_spectralFlux= [get_spectralFlux(x, hop_length=8)[0] for x in X_test]


## Re-Entrenamiento con descriptores de frecuencia

In [ ]:
knn_results = ClassifierKNN(X_train_spectralCentroid, Y_train, X_test_spectralCentroid, Y_test, 5)

Clasificador KNN de 5 vecinos
(19,)
(19,)
Entrenando con 93 muestras y probando con 19 muestras
Testing Accuracy = 52.63157894736842%
Matriz de Confusión:
[[9 1]
 [8 1]]
KNN - 3 vecinos
              precision    recall  f1-score   support

        kick       0.53      0.90      0.67        10
       snare       0.50      0.11      0.18         9

    accuracy                           0.53        19
   macro avg       0.51      0.51      0.42        19
weighted avg       0.52      0.53      0.44        19



In [ ]:
svm_results = ClassifierSVM(X_train_spectralCentroid, Y_train, X_test_spectralCentroid, Y_test)

Clasificador SVM
Entrenando con 93 muestras y probando con 19 muestras
Testing Accuracy = 52.63157894736842%
Matriz de Confusión:
[[10  0]
 [ 9  0]]
SVM
              precision    recall  f1-score   support

        kick       0.53      1.00      0.69        10
       snare       0.00      0.00      0.00         9

    accuracy                           0.53        19
   macro avg       0.26      0.50      0.34        19
weighted avg       0.28      0.53      0.36        19



Los descriptores de frecuencia no parecen ser tan útiles para distinguir los dos tipos de sonido pues el desempeño baja notablemente, se puede ver que ambos obtienen el mismo valor de accuracy (0.53) por lo que los desempeños parecen equivalentes, sin embargo, se puede ver que el resto de las métricas diferencian los modelos, tal parece que KNN con 5 vecinos tiene mejor recall para la clase "snare" pero la SVM tiene mejor recall para kick, se pueden ver algunas diferencias en el puntaje F1 de ambas también.

# Pregunta 6

### Clasificando timbres
En la carpeta 'Melodic' hay sonidos de instrumentos melódicos: 'Violin', 'Trumpet' y 'Sax'.
Utilizando los descriptores frecuenciales que considere pertinentes clasifique dichos sonidos.
Utilice las muestras de las carpeta 'train' para entrenar y las de la carpeta 'test' para evaluar.
Para cada clasificador entregue la matriz de confusión y la accuracy.

* Utilice KNN con la cantidad de vecinos que estime pertinentes y suba sus resultado
* Utilice SVM, con los parámetros que desee y suba sus resultados
* Utilizando MFCC como descriptores, clasifique utilizando KNN.

Los Mel Frequency Cepstral Coeﬃcients (Coeﬁcientes Cepstrales en las Frecuencias de Mel) o MFCCs son coeﬁcientes para la representación del habla basados en la percepción auditiva humana.

Comente los tres resultados obtenidos.

In [ ]:
X_violin_train_pre = [wavfile.read(data_path) for data_path in glob('Timbre/trainViolin/*.wav')]
X_violin_test_pre = [wavfile.read(data_path) for data_path in glob('Timbre/testViolin/*.wav')]

X_trumpet_train_pre = [wavfile.read(data_path) for data_path in glob('Timbre/trainTrumpet/*.wav')]
X_trumpet_test_pre = [wavfile.read(data_path) for data_path in glob('Timbre/testTrumpet/*.wav')]

X_sax_train_pre = [wavfile.read(data_path) for data_path in glob('Timbre/trainSax/*.wav')]
X_sax_test_pre = [wavfile.read(data_path) for data_path in glob('Timbre/testSax/*.wav')]

Como descriptor de frecuencia voy a volver a utilizar centroide espectral, además de agregar MFCC para le pregunta 3

In [ ]:
X_train_spectral_pre = [x[1].astype(float) for x in np.vstack((X_violin_train_pre, X_trumpet_train_pre, X_sax_train_pre))]
X_test_spectral_pre = [x[1].astype(float) for x in np.vstack([X_violin_test_pre, X_trumpet_test_pre, X_sax_test_pre])]

max_length = np.max([array.shape[0] for array in [x for x in X_train_spectral_pre+X_test_spectral_pre]])

X_train_mfcc  = np.vstack([mfcc(np.pad(x, (0,max_length-x.shape[0])).astype(float), sr = 44100).reshape(1,-1) for x in X_train_spectral_pre])
X_train_spectralCentroid  = np.vstack([spectral_centroid(np.pad(x, (0,max_length-x.shape[0])).astype(float), sr = 44100, n_fft=64,
                                                         hop_length=64)[0] for x in X_train_spectral_pre])

X_test_mfcc = np.vstack([mfcc(np.pad(x, (0,max_length-x.shape[0])).astype(float), sr = 44100).reshape(1,-1) for x in X_test_spectral_pre])
X_test_spectralCentroid = np.vstack([spectral_centroid(np.pad(x, (0,max_length-x.shape[0])).astype(float), sr = 44100, n_fft=64,
                                                       hop_length=64)[0] for x in X_test_spectral_pre])

Y_train = np.array(['violin']*len(X_violin_train_pre) + ['trumpet']*len(X_trumpet_train_pre) + ['sax']*len(X_sax_train_pre))
Y_test = np.array(['violin']*len(X_violin_test_pre) + ['trumpet']*len(X_trumpet_test_pre) + ['sax']*len(X_sax_test_pre))


### Descriptores de Frecuencia - Centroide Espectral

### KNN - 3 vecinos

In [ ]:
knn_results = ClassifierKNN(X_train_spectralCentroid, Y_train, X_test_spectralCentroid, Y_test, 3)

Clasificador KNN de 3 vecinos
(34,)
(34,)
Entrenando con 178 muestras y probando con 34 muestras
Testing Accuracy = 100.0%
Matriz de Confusión:
[[11  0  0]
 [ 0 12  0]
 [ 0  0 11]]
KNN - 3 vecinos
              precision    recall  f1-score   support

         sax       1.00      1.00      1.00        11
     trumpet       1.00      1.00      1.00        12
      violin       1.00      1.00      1.00        11

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### SVM - Parámetros por defecto

In [ ]:
svm_results = ClassifierSVM(X_train_spectralCentroid, Y_train, X_test_spectralCentroid, Y_test)

Clasificador SVM
Entrenando con 178 muestras y probando con 34 muestras
Testing Accuracy = 97.05882352941177%
Matriz de Confusión:
[[11  0  0]
 [ 0 12  0]
 [ 1  0 10]]
SVM
              precision    recall  f1-score   support

         sax       0.92      1.00      0.96        11
     trumpet       1.00      1.00      1.00        12
      violin       1.00      0.91      0.95        11

    accuracy                           0.97        34
   macro avg       0.97      0.97      0.97        34
weighted avg       0.97      0.97      0.97        34



### MFCC

### KNN - 3 vecinos

In [ ]:
knn_results = ClassifierKNN(X_train_mfcc, Y_train, X_test_mfcc, Y_test, 3)

Clasificador KNN de 3 vecinos
(34,)
(34,)
Entrenando con 178 muestras y probando con 34 muestras
Testing Accuracy = 100.0%
Matriz de Confusión:
[[11  0  0]
 [ 0 12  0]
 [ 0  0 11]]
KNN - 3 vecinos
              precision    recall  f1-score   support

         sax       1.00      1.00      1.00        11
     trumpet       1.00      1.00      1.00        12
      violin       1.00      1.00      1.00        11

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



### SVM - Parámetros por defecto

In [ ]:
svm_results = ClassifierSVM(X_train_mfcc, Y_train, X_test_mfcc, Y_test)

Clasificador SVM
Entrenando con 178 muestras y probando con 34 muestras
Testing Accuracy = 100.0%
Matriz de Confusión:
[[11  0  0]
 [ 0 12  0]
 [ 0  0 11]]
SVM
              precision    recall  f1-score   support

         sax       1.00      1.00      1.00        11
     trumpet       1.00      1.00      1.00        12
      violin       1.00      1.00      1.00        11

    accuracy                           1.00        34
   macro avg       1.00      1.00      1.00        34
weighted avg       1.00      1.00      1.00        34



# Pregunta 7
### Clasificando Estilos Musicales
En la carpeta 'Genres' hay sonidos de distintos géneros musicales.

Utilizando los descriptores que considere pertinentes y su clasificador preferido, clasifique dichos audios.

* Para cada estilo utilice el 80% de las muestras para entrenamiento y el 20% para testeo.

* Entregue la matriz de confusión y la accuracy.

Comente resultados de la matriz de confusión. ¿Para qué géneros funciona mejor?

In [ ]:
X_classical_pre = [wavfile.read(data_path) for data_path in glob('Genres/classical/*.wav')]
X_disco_pre = [wavfile.read(data_path) for data_path in glob('Genres/disco/*.wav')]
X_jazz_pre = [wavfile.read(data_path) for data_path in glob('Genres/jazz/*.wav')]
X_metal_pre = [wavfile.read(data_path) for data_path in glob('Genres/metal/*.wav')]
X_reggae_pre = [wavfile.read(data_path) for data_path in glob('Genres/reggae/*.wav')]

Como descriptor de frecuencia voy a volver a utilizar centroide espectral, además de agregar MFCC para le pregunta 3

In [ ]:
X = [x[1].astype(float) for x in np.vstack((X_classical_pre, X_disco_pre, X_jazz_pre, X_metal_pre, X_reggae_pre))]

max_length = np.max([array.shape[0] for array in [x[1] for x in X_classical_pre + X_disco_pre + X_jazz_pre + X_metal_pre + X_reggae_pre]])

X  = np.vstack([mfcc(np.pad(x, (0,max_length-x.shape[0])).astype(float), sr = 44100).reshape(1,-1) for x in X])
Y = np.array(['classical']*len(X_classical_pre) + ['disco']*len(X_disco_pre) + ['jazz']*len(X_jazz_pre) + \
                                            ['metal']*len(X_metal_pre) + ['reggae']*len(X_reggae_pre))


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 15)

### MFCC

### KNN - 3 vecinos

In [ ]:
# knn = KNeighborsClassifier(n_neighbors=3).fit(x_train, y_train)
knn_results = ClassifierKNN(x_train, y_train, x_test, y_test, 3)

Clasificador KNN de 3 vecinos
(32,)
(32,)
Entrenando con 128 muestras y probando con 32 muestras
Testing Accuracy = 68.75%
Matriz de Confusión:
[[6 0 0 0 0]
 [0 3 0 2 0]
 [2 0 7 0 0]
 [0 0 0 6 0]
 [0 0 6 0 0]]
KNN - 3 vecinos
              precision    recall  f1-score   support

   classical       0.75      1.00      0.86         6
       disco       1.00      0.60      0.75         5
        jazz       0.54      0.78      0.64         9
       metal       0.75      1.00      0.86         6
      reggae       0.00      0.00      0.00         6

    accuracy                           0.69        32
   macro avg       0.61      0.68      0.62        32
weighted avg       0.59      0.69      0.62        32

